## Objective

Create report to help us find opportunities to drive more sales and work more efficiently.

Here are some questions that we'd like to be able to answer:
- What days and times do we tend to be busiest? (calender/table -scale)
- How many pizzas are we making during peak periods? (table-scale)
- What are our best and worst selling pizzas?  (bar top and bottom 3)
- What's our average order value?
- How well are we utilizing our seating capacity? (we have 15 tables and 60 seats) (1 pizza = 1seat filled)(scater plot)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
from datetime import datetime, timedelta
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

### Import Data 

In [80]:
orders = pd.read_csv(r"E:\Data Projects\pizza_sales\orders.csv")
orders.head(2)

,order_id,date,time
0,1,2015-01-01,11:38:36
1,2,2015-01-01,11:57:40


In [79]:
order_details = pd.read_csv(r"E:\Data Projects\pizza_sales\order_details.csv")
order_details.head(2)

,order_details_id,order_id,pizza_id,quantity
0,1,1,hawaiian_m,1
1,2,2,classic_dlx_m,1


In [78]:
pizzas = pd.read_csv(r"E:\Data Projects\pizza_sales\pizzas.csv")
pizzas.head(2)

,pizza_id,pizza_type_id,size,price
0,bbq_ckn_s,bbq_ckn,S,12.75
1,bbq_ckn_m,bbq_ckn,M,16.75


In [77]:
pizza_types = pd.read_csv(r"E:\Data Projects\pizza_sales\pizza_types.csv", encoding='ISO-8859-1')
pizza_types.head(2)

,pizza_type_id,name,category,ingredients
0,bbq_ckn,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers,..."
1,cali_ckn,The California Chicken Pizza,Chicken,"Chicken, Artichoke, Spinach, Garlic, Jalapeno ..."


### Data Cleaning

In [84]:
print("Table: orders")
print("\nData Type:\n")
print(orders.dtypes)
print("")
print("Date and Time columns data types are not-correct.")
print("\n------------------------------------------\n")
print("Checking Null Values:")
print("")
print(orders.isnull().sum())
print("")
print("No null values present.")
print("\n------------------------------------------\n")
print("Any duplicates:", orders.duplicated().sum())

Table: orders

Data Type:

order_id     int64
date        object
time        object
dtype: object

Date and Time columns data types are not-correct.

------------------------------------------

Checking Null Values:

order_id    0
date        0
time        0
dtype: int64

No null values present.

------------------------------------------

Any duplicates: 0


In [83]:
print("Table: order_details")
print("\nData Type:\n")
print(order_details.dtypes)
print("")
print("All data types are correct!")
print("\n------------------------------------------\n")
print("Checking Null Values:")
print("")
print(order_details.isnull().sum())
print("")
print("No null values present.")
print("\n------------------------------------------\n")
print("Any duplicates:", order_details.duplicated().sum())

Table: order_details

Data Type:

order_details_id     int64
order_id             int64
pizza_id            object
quantity             int64
dtype: object

All data types are correct!

------------------------------------------

Checking Null Values:

order_details_id    0
order_id            0
pizza_id            0
quantity            0
dtype: int64

No null values present.

------------------------------------------

Any duplicates: 0


In [82]:
print("Table: pizzas")
print("\nData Type:\n")
print(pizzas.dtypes)
print("")
print("All data types are correct!")
print("\n------------------------------------------\n")
print("Checking Null Values:")
print("")
print(pizzas.isnull().sum())
print("")
print("No null values present.")
print("\n------------------------------------------\n")
print("Any duplicates:", pizzas.duplicated().sum())

Table: pizzas

Data Type:

pizza_id          object
pizza_type_id     object
size              object
price            float64
dtype: object

All data types are correct!

------------------------------------------

Checking Null Values:

pizza_id         0
pizza_type_id    0
size             0
price            0
dtype: int64

No null values present.

------------------------------------------

Any duplicates: 0


In [81]:
print("Table: pizza_types")
print("\nData Type:\n")
print(pizza_types.dtypes)
print("")
print("All data types are correct!")
print("\n------------------------------------------\n")
print("Checking Null Values:")
print("")
print(pizza_types.isnull().sum())
print("")
print("No null values present.")
print("\n------------------------------------------\n")
print("Any duplicates:", pizza_types.duplicated().sum())

Table: pizza_types

Data Type:

pizza_type_id    object
name             object
category         object
ingredients      object
dtype: object

All data types are correct!

------------------------------------------

Checking Null Values:

pizza_type_id    0
name             0
category         0
ingredients      0
dtype: int64

No null values present.

------------------------------------------

Any duplicates: 0


### Data Transformation

In [87]:
#change date columns data type to 'datetime'
orders.date = pd.to_datetime(orders.date)

In [88]:
#change time columns data type to 'timedelta'
orders.time = pd.to_datetime(orders.time)

In [89]:
# check data type, if changed.
orders.dtypes

order_id             int64
date        datetime64[ns]
time        datetime64[ns]
dtype: object

### Exploratory Data Analysis

We need:
- Merged Data
- Total Sales (quantity*price)
- Time of the day (Morning, After..)

In [93]:
#merge all tables
md = pd.merge(orders, order_details, on = 'order_id', how ='inner')
md = pd.merge(md, pizzas, on = 'pizza_id', how = 'inner')
md = pd.merge(md, pizza_types, on = 'pizza_type_id', how = 'inner')

md.head(2)

,order_id,date,time,order_details_id,pizza_id,quantity,pizza_type_id,size,price,name,category,ingredients
0,1,2015-01-01,2023-10-18 11:38:36,1,hawaiian_m,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,77,2015-01-02,2023-10-18 12:22:46,179,hawaiian_m,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"


In [94]:
# create a new 'total_sales' column
md['total_sales'] = md.price * md.quantity

# create a 'time_of_the_day' column
md['time_of_the_day'] = pd.cut(md.time.dt.hour, bins= [0,12,16,20,24],labels = ['Morning', 'Afternoon', 'Evening', 'Night'])

md.head(2)

,order_id,date,time,order_details_id,pizza_id,quantity,pizza_type_id,size,price,name,category,ingredients,total_sales,time_of_the_day
0,1,2015-01-01,2023-10-18 11:38:36,1,hawaiian_m,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",13.25,Morning
1,77,2015-01-02,2023-10-18 12:22:46,179,hawaiian_m,1,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",13.25,Morning


### a. Sales and Revenue Anlaysis

#### 1.  Total orders recieved?

In [95]:
total_orders = md.order_id.nunique()
print("Total orders:", total_orders)

Total orders: 21350


#### 2.  Total revenue generated in year 2015?

In [96]:
print("Total sales: $", md.total_sales.sum())

Total sales: $ 817860.05


#### 3.  Total pizzas sold?

In [97]:
print("Total pizzas sold:", md.quantity.sum())

Total pizzas sold: 49574


#### 4.  Plaoto's best and worst selling pizzas? 

In [98]:
# craete a table of pizzas in order by 'quantity_sold'
a4 = md.groupby('name').quantity.sum().reset_index().sort_values(by = 'quantity', ascending = False)

ac4 = px.bar(a4, x='quantity', y='name', color='quantity', height=500, width=850,
             color_continuous_scale = ['lightgrey','darkblue'], # set color theme
             labels= {'quantity':'Units Sold','name':'Pizza Name'}, # rename labels
             title = 'Pizza Power Rankings: Classic Deluxe Dominates, Mediterranean Pizza Struggles')

ac4.update_yaxes(title=None) # removing yaxes title

# add sub-title
ac4.update_layout(coloraxis_showscale=False, # remove color scale
                 annotations=[dict(
                     xref='paper', yref='paper',x=-0.344, y= 1.13,
                     showarrow=False, text='Pizzas by units sold',
                     font=dict(size=13,color='grey')
                 )])
# show chart
ac4.show()

#### 5.  Which pizzas should be prioritized for promotion?

In [99]:
# pizzas by total_sales
a51 = md.groupby('name').total_sales.sum().reset_index().sort_values(by = 'total_sales', ascending = False).head(8)

# pizzas by quantity_sold
a52 = md.groupby('name').quantity.sum().reset_index().sort_values(by = 'quantity', ascending = False).head(8)

# Find the common pizzas in both top 8 lists
common_pizzas = a51.merge(a52, on='name', how='inner')

ac5 = px.bar(
    common_pizzas, x='total_sales', y='name', color = 'total_sales',
    color_continuous_scale=['lightgrey','red'],
    title = 'Promotion-Ready Pizzas: Your Top Picks',
    labels={'total_sales':'Sales','name':'Pizza Name'},
    height = 450, width = 850
)
ac5.update_yaxes(title=None) # removing yaxes title

# add sub-title
ac5.update_layout(coloraxis_showscale=False, # remove color scale
                 annotations=[dict(
                     xref='paper',yref='paper',
                     x= -0.26,y=1.15,
                     text='Top common pizzas in revenue and units sold.',
                     showarrow = False,
                     font=dict(size=13, color='grey')
                 )])

ac5.show()

#### 6. Which pizzas should be considered for promotion as a secondary priority?

In [100]:
a6 = md.groupby('name').total_sales.sum().reset_index().sort_values(by = 'total_sales', ascending = False)

# Find pizzas from a51 that are not in common_pizzas
secondary_priority_pizzas = a6[~a6['name'].isin(common_pizzas['name'])].head(5).sort_index()

# chart
ac6 = px.bar(secondary_priority_pizzas, x='total_sales', y='name',
             color='total_sales', color_continuous_scale=['lightgrey','red'],
             title = 'Secondary Focus: Promoting the Second Best Pizzas',
             labels={'total_sales':'Sales','name':'Pizza Name'}, height = 450, width = 850)

ac6.update_yaxes(title=None) # remove y-axes title

# add sub-title
ac6.update_layout(coloraxis_showscale=False, # remove color scale
                 annotations=[dict(
                     xref='paper',yref='paper',
                     x= -0.27,y=1.15,
                     text='Secondary top pizzas by revenue.',
                     showarrow = False,
                     font=dict(size=13, color='grey')
                 )])

ac6.show()

#### 6.  Best and worst pizza categories by sales and units sold?

In [101]:
# units sold
a71 = md.groupby('category').quantity.sum().reset_index().sort_values(by = 'quantity', ascending = False)
# sales
a72 = md.groupby('category').total_sales.sum().reset_index().sort_values(by = 'total_sales', ascending = False)

# chart
ac7 = sp.make_subplots(rows=1, cols=2, subplot_titles=('Units Sold', 'Sales'), horizontal_spacing=0.15)

# colors
c71 = px.colors.sequential.Blues[::-1]
c72 = px.colors.sequential.Reds[::-1]

ac7.add_trace(go.Bar(
    x=a71.quantity, y=a71.category,
    name='Units Sold',
    orientation='h',
    marker=dict(color=c71)
), row=1, col=1)

# Add the least sold pizzas chart to the second column
ac7.add_trace(go.Bar(
    x=a72.total_sales, y=a72.category,
    name='Sales',
    orientation='h',
    marker=dict(color=c72)
), row=1, col=2)

# Update the layout and axis titles
ac7.update_layout(title='Dominant Duo: Classic and Supreme Pizzas Shine in <span style="color:blue">Units Sold</span> and <span style="color:red">Sales</span>',
                  height=500, width=980,
                  annotations=[dict(
                     showarrow=False,
                     font=dict(size=13,color='grey'),
                     text='Pizza category vs units sold and sales',
                     xref='paper',yref='paper',x=0.1,y=1.058
                 )], showlegend=False)

ac7.show()

#### 7.  Monthly trend: Categories by units sold?

In [102]:
a7 = md.groupby([md.date.dt.strftime('%m'),'category']).total_sales.sum().reset_index()
# rename month names
a7['date'] = a7['date'].replace({
    '01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr', '05': 'May', '06': 'Jun',
    '07': 'Jul', '08': 'Aug', '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'
})

# Create a custom color scale
line_colors = {'Classic': 'red', 'Chicken': 'lightgrey', 'Supreme': 'lightgrey', 'Veggie': 'lightgrey'}

# Line chart
ac7 = px.line(a7, x='date', y='total_sales', color='category', color_discrete_map=line_colors,
              labels={'date':'Month', 'total_sales':'Sales', 'category':'Category'},
              title = "Pizza Category Performance Over Months: <span style='color: red'>Classic</span> Stays on Top")
ac7.update_layout(height=450, width=900,
                  annotations=[
                      dict(text='Category monthly trend by total sales',
                           xref='paper', yref='paper',
                           x=-0.062, y=1.14,
                           showarrow=False,
                           font=dict(size=13, color='gray'))
                  ])

ac7.update_xaxes(title=None) # remove xaxes title

# Remove the legend
ac7.update_traces(showlegend=False)

ac7.show()

#### 8.  Is purchase behaviour and sales influenced by pizza price?
Any relation between pizza price, quantity_sold and total_sales generated?

In [103]:
pizza_sales = md.groupby('pizza_id').agg({
    'price': 'first',  # Get the original price (assuming it's the same for a given pizza)
    'total_sales': 'sum',  # Calculate the total sales for each pizza
    'quantity': 'sum'  # Calculate the total quantity sold for each pizza
}).reset_index()

#rename columns
pizza_sales.rename(columns={'price': 'pizza_price', 'quantity': 'total_quantity_sold'}, inplace=True)

# Create subplots
ac8 = sp.make_subplots(rows=1, cols=2)

# Scatter plot in Column 1
scatter1 = px.scatter(x=pizza_sales.pizza_price, y=pizza_sales.total_quantity_sold, color_discrete_sequence=['darkblue'],
                     labels={'x': 'Pizza Price', 'y': 'Units Sold'})
ac8.add_trace(scatter1.data[0], row=1, col=1)

# Add a trendline
trendline1 = np.polyfit(pizza_sales.pizza_price, pizza_sales.total_quantity_sold, 1)
trendline_values1 = np.polyval(trendline1, pizza_sales.pizza_price)
scatter1_trendline = px.line(x=pizza_sales.pizza_price, y=trendline_values1, line_shape='linear')
#scatter2_trendline.data[0].line.color = 'darkblue'  # Change the trendline color to green
ac8.add_trace(scatter1_trendline.data[0], row=1, col=1)

# Scatter plot in Column 2
scatter2 = px.scatter(x=pizza_sales.pizza_price, y=pizza_sales.total_sales, color_discrete_sequence=['red'],
                     labels={'x': 'Pizza Price', 'y': 'Sales'})
ac8.add_trace(scatter2.data[0], row=1, col=2)

# Add a trendline
trendline2 = np.polyfit(pizza_sales.pizza_price, pizza_sales.total_sales, 1)
trendline_values2 = np.polyval(trendline2, pizza_sales.pizza_price)
scatter2_trendline = px.line(x=pizza_sales.pizza_price, y=trendline_values2, line_shape='linear')
scatter2.update_traces(
    line=dict(color='green'),  # Change the trendline color to green
    name='Trendline 1',
    mode='lines'
)
ac8.add_trace(scatter2_trendline.data[0], row=1, col=2)

# Configure layout
ac8.update_layout(
    annotations=[dict(text='Relationship between Pizza Price and Purchase Behavior',
                     font=dict(size=13,color='gray'),
                      xref='paper',yref='paper',x=-0.039,y=1.14,
                     showarrow=False)],
    yaxis_title='Units Sold',
    yaxis2_title='Sales',
    xaxis_title='Pizza Price',
    xaxis2_title='Pizza Price',
    title_text = 'Pizza Price vs. Customer Behavior: Steady Units Sold, Limited Impact on Sales',
    showlegend=False,  # Hides the default legends,
    height=500
)

# Show the plot
ac8.show()

#### 9. Average price per each category vs quantity sold relation? 

In [104]:
a9 = md.groupby('category').agg({'price':'mean','quantity':'sum'}).reset_index().sort_values(by='quantity',ascending=False)

# Create a scatter plot without text
ac9 = px.scatter(x=a9.price, y=a9.quantity,
                 labels={'x': 'Category Average Price', 'y': 'Units Sold'},
                 color_discrete_sequence=['darkblue'],
                 height=450, width=650,
                 trendline='ols',
                 trendline_color_override='blue',
                 title='Units Sold Drops as Category Prices Rise')

# Add a text trace with category labels
ac9.add_trace(go.Scatter(x=a9.price, y=a9.quantity, text=a9.category, 
                        mode='text', showlegend=False, textposition='top center'))

ac9.update_layout(
    annotations=[dict(
        text='Category avg-price($) vs units sold',
        xref='paper',yref='paper',x=-0.1,y=1.16,
        font=dict(size=13,color='gray'),
        showarrow=False
    )]
)

ac9.show()


### b. Customer Behavior Analysis

#### 1. Average order quantity a customer orders? 

In [105]:
b11 = md.groupby('order_id').quantity.sum().reset_index()
b11.rename(columns={'pizzas_per_order':'quantity'},inplace=True)
b11['pizzas_per_order']=b11['quantity'].map(lambda x : '2+' if x>2 else '1-2')

b1 = b11.groupby('pizzas_per_order').count().reset_index()

# pie chart
bc1 = px.pie(
    b1, names='pizzas_per_order',values='order_id', # give pie chart values
    color_discrete_sequence=['rgb(0,0,102)', 'rgb(0,0,153)'], # assign colors
    labels={'order_id':'Order Count','pizzas_per_order':'Pizzas Per Order'}, # give labels for hover
    title='Pizza Ordering Habits: Majority Choose 1-2 Pizzas Per Order', height=500, width=500
)


bc1.update_traces(textposition='inside', textinfo='percent+label', hole=0.6) #display the info on pie

# Hide the legend
bc1.update_layout(showlegend=False,
                  annotations=[dict(
                      text='Pizzas Per Order vs Order Count',
                      xref='paper', yref='paper',
                      x=-0.165, y=1.13, showarrow=False,
                      font=dict(size=13, color='gray')
                  )]
                 )

bc1


#### 2. Average price a customer spends on an order (average order value).

In [106]:
b2 = md.groupby('order_id').total_sales.sum().mean()
print(f"Average Order Value: ${b2:.2f}")

Average Order Value: $38.31


#### 3. What's the order count in peak hours.

In [107]:
b3= md.groupby(md.time.dt.hour).order_id.nunique().sort_values(ascending=False).reset_index()

print(f"Max order count in peak hours: {b3.order_id.max()} orders")

Max order count in peak hours: 2520 orders


#### 4. Which pizza size is most ordered?

In [108]:
b4 = md.groupby('size').quantity.sum().reset_index()

bc4 = px.pie(
    b4,values='quantity',names='size', height=500,width=500,
    title='Most Popular Pizza Size: Large Takes the Crown',
    labels={'size':'Pizza Size','quantity':'Units Sold'},
    color_discrete_sequence=['#1B03A3', '#1432BA', '#0E61D1','#0790E8','#00BFFF']
)

bc4.update_layout(
                  annotations=[dict(
                      text='Pizza Sizes vs Units Sold',
                      xref='paper', yref='paper',
                      x=-0.168, y=1.13, showarrow=False,
                      font=dict(size=13, color='gray')
                  )]
                 )


bc4.show()

#### 5. Does price of pizza size affect purchase behavior?

In [109]:
size_mapping = {'S': 1, 'M': 2, 'L': 3, 'XL': 4, 'XXL': 5}
sizecorr = md.copy()
sizecorr['size'] = sizecorr['size'].map(size_mapping)

b5 = sizecorr.groupby('size').agg({'quantity':'sum', 'price':'mean'}).reset_index()

# relation between the quantity ordred of size and price
#b5.corr()

bc5 = px.scatter(b5, x='size', y = 'quantity', trendline='ols', width=650, height=450,
                labels={'size':'Pizza Size', 'quantity':'Units Sold'},
                color_discrete_sequence=['darkblue'],
                trendline_color_override='blue',
                title='Elevated Pizza Size Prices Lead to Reduced Unit Sold')

# Define custom tick values for the x-axis
bc5.update_xaxes(title=None,
                 tickmode='array',
                 tickvals=list(size_mapping.values()),
                 ticktext=list(size_mapping.keys())
                )
bc5.update_yaxes(title=None)
bc5.update_layout(
                  annotations=[dict(
                      text='Pizza Sizes vs Units Sold',
                      xref='paper', yref='paper',
                      x=-0.1, y=1.14, showarrow=False,
                      font=dict(size=13, color='gray')
                  )]
                 ).show()

### c. Operational Efficiency Analysis

#### 1. What days and times do we tend to be busiest? 

In [110]:
c1 = md.groupby([md.date.dt.strftime('%A'),'time_of_the_day']).order_id.nunique().reset_index()
c1.rename(columns={'date':'day','order_id':'order_count'}, inplace=True)

# Create a pivot table
pivot_table = c1.pivot_table(values='order_count', index='time_of_the_day', columns='day')

# Create a heatmap using Plotly
cc1 = px.imshow(pivot_table, color_continuous_scale='Blues', height=500, width=1000,
                title = 'Busiest Day and Times: Fridays, Afternoons, and Evenings Stand Out',
               labels={'day':'Day','time_of_the_day':'Time'})

# Remove the titles on x-axis and y-axis
cc1.update_xaxes(title=None)
cc1.update_yaxes(title=None)

# update layout
cc1.update_layout(annotations=[dict(
    text='Day of Week vs Time of the Day',
    xref='paper', yref='paper',
    x=-0.038, y=1.13, showarrow=False,
    font=dict(size=13, color='gray')
)])

# Customize hover labels
cc1.update_traces(hovertemplate="Day: %{x}<br>Time of the day: %{y}<br>Orders: %{z}")

# Show the plot
cc1.show()

#### 2. Which hour is the most busiest?

In [111]:
c2 = md.groupby(md.time.dt.hour).order_id.nunique().reset_index()

# hourly order_count
cc2 = px.bar(
    c2, x='time', y='order_id',
    labels={'time': 'Hour', 'order_id': 'Order Count'},
    title='12 p.m. Marks as the Peak Hour',
    color='order_id', height=450, width=950,
    color_continuous_scale=['lightgrey', 'darkblue']
)

cc2.update_layout(
    coloraxis_showscale=False,
    annotations=[
        dict(
            text='Hourly Order Count Trend',
            xref='paper', yref='paper',
            x=-0.04, y=1.14,
            showarrow=False,
            font=dict(size=13, color='gray')
        )
    ]
)

cc2.show()

#### 3. Which month is the most and least busiest?

In [112]:
c3 = md.groupby(md.date.dt.strftime('%m')).order_id.count().reset_index()
c3['date'] = c3['date'].replace({
    '01': 'Jan', '02': 'Feb', '03': 'Mar', '04': 'Apr', '05': 'May', '06': 'Jun',
    '07': 'Jul', '08': 'Aug', '09': 'Sep', '10': 'Oct', '11': 'Nov', '12': 'Dec'
})

#chart
cc3= px.bar(c3, x='date', y='order_id',
    color='order_id',
    color_continuous_scale=['lightgrey','darkblue'],
    labels={'date':'Month','order_id':'Order Count'},
    title='July, May, and November, March, and January Impress, While October Struggles',
    height=450, width=950
)

cc3.update_xaxes(title=None)

cc3.update_layout(
    coloraxis_showscale=False,
    annotations=[dict(
        xref='paper',yref='paper',
        x=-0.045,y=1.15,
        showarrow=False,
        text='Order count vs Month',
        font=dict(size=13,color='grey')
    )]
)
cc3.show()

#### 4. How many pizzas are made in peak hours?

In [113]:
c4= md.groupby(md.time.dt.hour).quantity.sum().sort_values(ascending=False).reset_index()

print(f"Max pizzas made in peak hour: {c4.quantity.max()} Pizzas\n")

Max pizzas made in peak hour: 6776 Pizzas



#### 5. How well are we utilizing our seating capacity? (15 Tables and 60 seats)

Assumptions:

- A person dines for 1 hour
- 1 pizza = 1 person
- 4 pizzas order = 4 seats filled
- 1 order = 1 table filled

In [114]:
order_i = orders.merge(order_details,on='order_id',how='left')
order_in = order_i.groupby(['order_id','date','time']).quantity.sum().reset_index()

order_in['table_no'] = pd.cut(order_in.quantity, bins=[0,4,8,12,16,20,24,28,32], labels=[1,2,3,4,5,6,7,8])

order_in['time'] = order_in['time'].dt.time

order_in.head(2)

,order_id,date,time,quantity,table_no
0,1,2015-01-01,11:38:36,1,1
1,2,2015-01-01,11:57:40,5,2


In [115]:
order_out = order_in.copy()
order_out.quantity = order_out.quantity.apply(lambda x: -x)
order_out.table_no = order_out.table_no.apply(lambda x: -x)
order_out['time'] = order_out['time'].apply(lambda x: datetime.combine(datetime.min, x) + timedelta(hours=1))
order_out['time'] = order_out['time'].apply(lambda x: x.time())

order_out.head(2)

,order_id,date,time,quantity,table_no
0,1,2015-01-01,12:38:36,-1,-1
1,2,2015-01-01,12:57:40,-5,-2


In [116]:
total_orders = pd.concat([order_in, order_out])

total_orders.head(2)

,order_id,date,time,quantity,table_no
0,1,2015-01-01,11:38:36,1,1
1,2,2015-01-01,11:57:40,5,2


In [117]:
# Sort the DataFrame by date and time
total_orders.sort_values(by=['date', 'time'], inplace=True)

# Reset the index to sequential integers
total_orders.reset_index(drop=True, inplace=True)

# Calculate the cumulative sum of 'table_no' and 'quantity' per date
total_orders['total_running_tables'] = total_orders.groupby('date')['table_no'].cumsum()
total_orders['total_running_customers'] = total_orders.groupby('date')['quantity'].cumsum()
total_orders.head(20)

,order_id,date,time,quantity,table_no,total_running_tables,total_running_customers
0,1,2015-01-01,11:38:36,1,1,1,1
1,2,2015-01-01,11:57:40,5,2,3,6
2,3,2015-01-01,12:12:28,2,1,4,8
3,4,2015-01-01,12:16:31,1,1,5,9
4,5,2015-01-01,12:21:30,1,1,6,10
5,6,2015-01-01,12:29:36,2,1,7,12
6,1,2015-01-01,12:38:36,-1,-1,6,11
7,7,2015-01-01,12:50:37,1,1,7,12
8,8,2015-01-01,12:51:37,1,1,8,13
9,9,2015-01-01,12:52:01,9,3,11,22


In [118]:
total_orders['table_exceeding'] = total_orders.total_running_tables.map(lambda x: 'No' if x<16 else 'Yes')
ee = total_orders
ee.head(2)

,order_id,date,time,quantity,table_no,total_running_tables,total_running_customers,table_exceeding
0,1,2015-01-01,11:38:36,1,1,1,1,No
1,2,2015-01-01,11:57:40,5,2,3,6,No


In [119]:
eec= px.scatter(ee, x='date', y='total_running_tables',
                title='Crowded Dine-In: Running Out of Tables',
                color='table_exceeding', color_discrete_map={'Yes':'red','No':'darkblue'},
                labels={'date':'Date','total_running_tables':'Total Running Tables','table_exceeding':'Table Exceeding'})
eec.update_xaxes(title=None)
eec.update_layout(
    annotations=[dict(
        xref='paper',yref='paper',
        x=-0.042,y=1.13,
        showarrow=False,
        text='Tables Running vs Date',
        font=dict(size=13,color='grey')
    )]
)

eec.show()